In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from collections import Counter
from sklearn.decomposition import PCA

In [2]:
from xgboost import XGBClassifier
test = pd.read_csv('../output_preprocessed/test_last_transaction.csv')
test_features = test[test.columns[2:]]

predictions = pd.DataFrame()
predictions['msno']=test['msno']

C:\Users\romai\Miniconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
train = pd.read_csv('../output_preprocessed/train_last_transaction.csv')
train = train[train.columns[1:]]
train_features = train[train.columns[1:]]
train_labels = train["is_churn"]
del train

In [4]:
train_features.head()

,trans_count,logs_count,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,membership_expire_date,is_cancel,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,2,0.0,18.0,36.0,2.0,9.0,20050406.0,0.0,0.0,0.0,...,0.0,0.0,20170121.0,4.0,0.0,2.0,5.0,76.0,74.0,19799.702
1,23,1.0,10.0,38.0,1.0,9.0,20050407.0,0.0,0.0,0.0,...,0.0,0.0,20170319.0,0.0,0.0,0.0,0.0,1.0,1.0,281.600
2,10,0.0,11.0,27.0,2.0,9.0,20051016.0,0.0,0.0,0.0,...,0.0,0.0,20161225.0,61.0,21.0,9.0,11.0,44.0,130.0,15845.692
3,3,5.0,13.0,23.0,2.0,9.0,20051102.0,40.0,30.0,149.0,...,20170426.0,0.0,20170331.0,28.0,4.0,5.0,4.0,19.0,51.0,6171.145
4,9,17.0,3.0,27.0,1.0,9.0,20051228.0,38.0,90.0,477.0,...,20170528.0,0.0,20170331.0,0.0,0.0,0.0,2.0,11.0,12.0,3132.042


In [5]:
(train_labels == 1).sum()/len(train_labels)

0.076786848149922787

In [6]:
train_features.columns

Index(['trans_count', 'logs_count', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel', 'date',
       'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq',
       'total_secs'],
      dtype='object')

In [7]:
def replacemean(x, t, mean):
    if(x < t):
        return mean
    else:
        return x

In [8]:
mean_registration_init_time = train_features[train_features["registration_init_time"] > 1]["registration_init_time"].mean()
train_features["registration_init_time"] = train_features["registration_init_time"].apply(lambda x: replacemean(x, 1, mean_registration_init_time))
test_features["registration_init_time"] = test_features["registration_init_time"].apply(lambda x: replacemean(x, 1, mean_registration_init_time))

C:\Users\romai\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
mean_transaction_date = train_features[train_features["transaction_date"] > 1]["transaction_date"].mean()
train_features["transaction_date"] = train_features["transaction_date"].apply(lambda x: replacemean(x, 1, mean_transaction_date))
test_features["transaction_date"] = test_features["transaction_date"].apply(lambda x: replacemean(x, 1, mean_transaction_date))

C:\Users\romai\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
mean_date = train_features[train_features["date"] > 1]["date"].mean()
train_features["date"] = train_features["date"].apply(lambda x: replacemean(x, 1, mean_date))
test_features["date"] = test_features["date"].apply(lambda x: replacemean(x, 1, mean_date))

C:\Users\romai\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
cols_to_transform = ["city", "gender", "payment_method_id"]

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train_features.loc[:][cols_to_transform])

ONE_HOT_train = (enc.transform(train_features.loc[:][cols_to_transform]).toarray()).transpose()
ONE_HOT_test = (enc.transform(test_features.loc[:][cols_to_transform]).toarray()).transpose()

for col in cols_to_transform:
    del train_features[col]
    del test_features[col]

for i in range(0, ONE_HOT_train.shape[0]):
    train_features["ONE_HOT_"+str(i)] = ONE_HOT_train[i]
    test_features["ONE_HOT_"+str(i)] = ONE_HOT_test[i]

C:\Users\romai\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm.fit(train_features)

train_features_preprocessed = norm.transform(train_features)
test_features_preprocessed = norm.transform(test_features)

In [13]:
def compute_logloss(pred_proba, ischurn):
    logloss = -((ischurn*np.log(pred_proba)).sum() + ((1 - ischurn)*np.log(1 - pred_proba)).sum())
    return (logloss / len(pred_proba))

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn import ensemble
# from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb

# MAX_FOLD = 10
# logloss_gbm = 0
# logloss_lgbm = 0
# logloss_adb = 0
# logloss_lg = 0
# for i in range(0, MAX_FOLD):
    
#     #gbm = XGBClassifier(n_estimators=500, learning_rate=0.025, max_depth=4, subsample=0.8, colsample_bytree=1)
# #     params = {
# #         'learning_rate' : 0.1,
# #          'n_estimators' : 1000,
# #          'max_depth' : 10,
# #          'min_child_weight': 12,
# #          'gamma' :10,
# #          'subsample':0.8,
# #          'colsample_bytree':0.8,
# #          'objective': 'binary:logistic',
# #          'scale_pos_weight':1,
# #          'seed':27
# #     }
    
#     gbm = XGBClassifier(max_delta_step = 1)
#     lgbm = lgb.LGBMClassifier()
#     adb = ensemble.AdaBoostClassifier()
#     lg = LogisticRegression()
    
#     X_train, X_test, y_train, y_test = train_test_split(train_features_preprocessed, train_labels, test_size = 0.1)
#     print("COMPUTING FOLD "+str(i)+"/"+str(MAX_FOLD))
    
#     params = {
#         'eta': 0.02, #use 0.002
#         'max_depth': 7,
#         'objective': 'binary:logistic',
#         'eval_metric': 'logloss',
#         'seed': i,
#         'silent': True
#     }
#     gbm.fit(X_train, y_train)
#     lgbm.fit(X_train, y_train)
#     adb.fit(X_train, y_train)
#     lg.fit(X_train, y_train)
    
#     pred_gbm = gbm.predict_proba(X_test)[:,1]
#     logloss_gbm_tmp = compute_logloss(pred_gbm, y_test)
#     logloss_gbm += logloss_gbm_tmp
    
#     pred_lgbm = lgbm.predict_proba(X_test)[:,1]
#     logloss_lgbm_tmp = compute_logloss(pred_lgbm, y_test)
#     logloss_lgbm += logloss_lgbm_tmp
        
#     pred_adb = adb.predict_proba(X_test)[:,1]
#     logloss_adb_tmp = compute_logloss(pred_adb, y_test)
#     logloss_adb += logloss_adb_tmp
    
#     pred_lg = lg.predict_proba(X_test)[:,1]
#     logloss_lg_tmp = compute_logloss(pred_lg, y_test)
#     logloss_lg += logloss_lg_tmp
    
#     print(int(10000*logloss_gbm_tmp)/10000,
#           int(10000*logloss_lgbm_tmp)/10000,
#           int(10000*logloss_lg_tmp)/10000,
#           int(10000*logloss_adb_tmp)/10000)
    
#     print(compute_logloss(np.exp(np.mean([pred_gbm.apply(lambda x: np.log(x)),\
#                                         pred_lgbm.apply(lambda x: np.log(x)),\
#                                         pred_adb.apply(lambda x: np.log(x)),\
#                                         pred_lg.apply(lambda x: np.log(x)),\
#                                         ], axis =0)), y_test))
    
#     del y_test, X_train, X_test, y_train

# print("END")
# print(logloss_gbm/MAX_FOLD, logloss_lgbm/MAX_FOLD, logloss_adb/MAX_FOLD, logloss_lg/MAX_FOLD)

In [ ]:
from xgboost import XGBClassifier
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

gbm = XGBClassifier(max_delta_step = 1)
lgbm = lgb.LGBMClassifier()
adb = ensemble.AdaBoostClassifier()
lg = LogisticRegression()
    
gbm.fit(train_features_preprocessed, train_labels)
lgbm.fit(train_features_preprocessed, train_labels)
adb.fit(train_features_preprocessed, train_labels)
lg.fit(train_features_preprocessed, train_labels)
    
pred_gbm = gbm.predict_proba(test_features_preprocessed) 
pred_lgbm = lgbm.predict_proba(test_features_preprocessed)      
pred_adb = adb.predict_proba(test_features_preprocessed)
pred_lg = lg.predict_proba(test_features_preprocessed)

In [ ]:
k1 = pd.read_csv('../from_kaggle_kernels/kk_pred.csv')
k2 = pd.read_csv('../from_kaggle_kernels/submission.csv')
k3 = pd.read_csv('../from_kaggle_kernels/xgbsub.csv')

In [ ]:
predictions["pred_gbm"] = pred_gbm
predictions["pred_lgbm"] = pred_lgbm
predictions["pred_adb"] = pred_adb
predictions["pred_lg"] = pred_lg

predictions['is_churn'] = np.exp(np.mean(
        [
        k1['is_churn'].apply(lambda x: np.log(x)),\
        k2['is_churn'].apply(lambda x: np.log(x)),\
        k3['is_churn'].apply(lambda x: np.log(x)),\
        predictions["pred_gbm"].apply(lambda x: np.log(x)),\
        predictions["pred_lgbm"].apply(lambda x: np.log(x)),\
        predictions["pred_adb"].apply(lambda x: np.log(x)),\
        predictions["pred_lg"].apply(lambda x: np.log(x)),\
        ], axis =0))

del predictions["pred_gbm"]
del predictions["pred_lgbm"]
del predictions["pred_adb"]
del predictions["pred_lg"]

predictions.to_csv("../output/from_kaggle_kernels_and_calculs.csv", float_format='%.6f', index = False)